# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,sodankyla,67.4167,26.6000,8.45,74,32,3.06,FI,1696167494
1,1,1,sambava,-14.2667,50.1667,25.24,74,51,3.51,MG,1696167495
2,2,2,grytviken,-54.2811,-36.5092,4.63,84,57,4.14,GS,1696167495
3,3,3,kurchatov,51.6601,35.6521,22.23,36,53,4.94,RU,1696167431
4,4,4,balaklava,44.5000,33.6000,21.66,51,97,3.19,UA,1696167495


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)
 

# Display the map

all_city_map
 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud


# Drop any rows with null values
city_ideal = city_cloud.dropna()



# Display sample data
city_ideal



,City_ID,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,48,fort bragg,35.1390,-79.0060,21.57,88,0,2.57,US,1696167505
55,55,55,haiku-pauwela,20.9219,-156.3051,25.58,77,0,2.57,US,1696167507
118,118,118,todos santos,23.4500,-110.2167,26.28,90,0,2.96,MX,1696167521
127,127,127,saint-pierre,-21.3393,55.4781,25.82,61,0,2.57,RE,1696167523
165,165,165,mashhad,36.2970,59.6062,26.08,27,0,3.09,IR,1696167531
285,285,285,laurel,39.0993,-76.8483,21.46,80,0,3.09,US,1696167476
327,327,327,puerto penasco,31.3167,-113.5333,23.22,46,0,1.84,MX,1696167567
345,345,345,balkanabat,39.5108,54.3671,25.99,15,0,2.28,TM,1696167571
350,350,350,santiago de cao,-7.9589,-79.2392,21.07,78,0,4.12,PE,1696167572
427,427,427,monor,47.3513,19.4473,23.00,40,0,4.12,HU,1696167589


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df



,City,Country,Lat,Lng,Humidity,Hotel Name
48,fort bragg,US,35.1390,-79.0060,88,
55,haiku-pauwela,US,20.9219,-156.3051,77,
118,todos santos,MX,23.4500,-110.2167,90,
127,saint-pierre,RE,-21.3393,55.4781,61,
165,mashhad,IR,36.2970,59.6062,27,
285,laurel,US,39.0993,-76.8483,80,
327,puerto penasco,MX,31.3167,-113.5333,46,
345,balkanabat,TM,39.5108,54.3671,15,
350,santiago de cao,PE,-7.9589,-79.2392,78,
427,monor,HU,47.3513,19.4473,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
todos santos - nearest hotel: Hotel Casa Tota
saint-pierre - nearest hotel: Tropic Hotel
mashhad - nearest hotel: هتل تعاون
laurel - nearest hotel: Quality Inn & Suites
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
balkanabat - nearest hotel: Nebitçi
santiago de cao - nearest hotel: Hostal El Encanto
monor - nearest hotel: Nyerges Hotel Termál*** Superior
campo gallo - nearest hotel: Hotel La Plaza
siniscola - nearest hotel: Agriturismo Sa Costa
ojinaga - nearest hotel: Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
48,fort bragg,US,35.1390,-79.0060,88,Airborne Inn Lodging
55,haiku-pauwela,US,20.9219,-156.3051,77,Inn At Mama's Fish House
118,todos santos,MX,23.4500,-110.2167,90,Hotel Casa Tota
127,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
165,mashhad,IR,36.2970,59.6062,27,هتل تعاون
285,laurel,US,39.0993,-76.8483,80,Quality Inn & Suites
327,puerto penasco,MX,31.3167,-113.5333,46,Hotel Paraíso Del Desierto
345,balkanabat,TM,39.5108,54.3671,15,Nebitçi
350,santiago de cao,PE,-7.9589,-79.2392,78,Hostal El Encanto
427,monor,HU,47.3513,19.4473,40,Nyerges Hotel Termál*** Superior


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)